# Crawl and Save Tiki Product Comments
This notebook demonstrates how to crawl product comments from Tiki.vn and save them to a JSON file.

In [1]:
import requests
import pandas as pd
import json
import re

cookies = {
    'TIKI_GUEST_TOKEN': 'oC3QPaXrT4tZnw9jHfLiJhUvMmxSbEyN',
    'TOKENS': '{%22access_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1763654224277%2C%22guest_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj4bc.1k.11.2l',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj2q9.3.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605977607.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605977607.0',
    '_ga': 'GA1.2.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzkzizhzkzlznzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkzlzjzgzjzdzizizdzizlzjzmzqzkzlzjzgzjzdzizlzjzmzqzkzlzjzgzjzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605977598%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjAbsoluteSessionInProgress': '0',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605978058486',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    'cto_bundle': '7L6ha19NVXNkQmJ6aEVLcXNqbHdjcVZoQ0kzTUZwcEMyNCUyRm5nV3A2SThuOGxTRjI4Wlk1NU9xRnBEOG9tUjd2ekhyZEQxeE9qaVQ4MnFpbiUyRllGd2JiQUpTMW94MlNsTnYxd3dOYWtRcXhGdDNxSjdBVmNxU0FnUSUyQjlWYjhqTUtLdVl2cTBheWFvS0ZnY2pLdlpWRlEyUFF0Y1ElM0QlM0Q',
    'TIKI_RECENTLYVIEWED': '58259141',
    '_ants_event_his': '%7B%22action%22%3A%22view%22%2C%22time%22%3A1605977607258%7D',
    '_hjIncludedInPageviewSample': '1',
    '_hjIncludedInSessionSample': '1',
    '_gat': '1',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

url = input("Enter Tiki product link: ")
m = re.search(r'-p(\d+)\.html', url)
if m:
    product_id = m.group(1)
    product_api = f'https://tiki.vn/api/v2/products/{product_id}'
    resp = requests.get(product_api, headers=headers, cookies=cookies)
    if resp.status_code == 200:
        data = resp.json()
        seller_id = data.get('current_seller', {}).get('id')
        # Try to get spid from URL or API response
        spid = None
        m_spid = re.search(r'[?&]spid=(\d+)', url)
        if m_spid:
            spid = m_spid.group(1)
        elif 'configurable_options' in data and data['configurable_options']:
            spid = data['configurable_options'][0]['values'][0].get('id')
        elif 'id' in data.get('current_seller', {}):
            spid = data['current_seller']['id']
        if not spid:
            print("Warning: Could not determine spid, API may not return comments correctly.")
else:
    raise ValueError("Could not extract product_id from the URL. Please check the link.")

params = {
    'limit': '5',
    'include': 'comments,contribute_info,attribute_vote_summary',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'product_id': product_id,
    'spid': spid,
}

def comment_parser(json):
    d = dict()
    d['id'] = json.get('id')
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    d['thank_count'] = json.get('thank_count')
    d['customer_id']  = json.get('customer_id')
    d['rating'] = json.get('rating')
    d['created_at'] = json.get('created_at')
    created_by = json.get('created_by', {})
    d['customer_name'] = created_by.get('name')
    d['purchased_at'] = created_by.get('purchased_at')
    return d

result = []
valid_count = 0
max_valid = 50
print(f'Crawl comment for product {product_id} (spid={spid})')
for i in range(1, 100):  # Increase page range to ensure enough valid comments
    if valid_count >= max_valid:
        break
    params['page'] = i
    response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params, cookies=cookies)
    if response.status_code == 200:
        print(f'Crawl comment page {i} success!!!')
        for comment in response.json().get('data', []):
            if comment.get('content', '').strip():
                result.append(comment_parser(comment))
                valid_count += 1
                if valid_count >= max_valid:
                    break
    else:
        print(f'Failed to crawl page {i}, status code: {response.status_code}')

df_comment = pd.DataFrame(result)

# Save to JSON file
df_comment[['id', 'content']].to_json('comments_data_ncds.json', orient='records', force_ascii=False, indent=2)

# Debug: Show first few rows of the DataFrame and file content
print("First 5 rows of DataFrame:")
print(df_comment[['id', 'content']].head())
with open('comments_data_ncds.json', encoding='utf-8') as f:
    print("\nFirst 500 chars of saved JSON file:")
    print(f.read(500))

Crawl comment for product 1640153 (spid=10862996)
Crawl comment page 1 success!!!
Crawl comment page 1 success!!!
Crawl comment page 2 success!!!
Crawl comment page 2 success!!!
Crawl comment page 3 success!!!
Crawl comment page 3 success!!!
Crawl comment page 4 success!!!
Crawl comment page 4 success!!!
Crawl comment page 5 success!!!
Crawl comment page 5 success!!!
Crawl comment page 6 success!!!
Crawl comment page 6 success!!!
Crawl comment page 7 success!!!
Crawl comment page 7 success!!!
Crawl comment page 8 success!!!
Crawl comment page 8 success!!!
Crawl comment page 9 success!!!
Crawl comment page 9 success!!!
Crawl comment page 10 success!!!
Crawl comment page 10 success!!!
Crawl comment page 11 success!!!
Crawl comment page 11 success!!!
Crawl comment page 12 success!!!
Crawl comment page 12 success!!!
Crawl comment page 13 success!!!
Crawl comment page 13 success!!!
Crawl comment page 14 success!!!
Crawl comment page 14 success!!!
Crawl comment page 15 success!!!
Crawl comme

In [5]:
import requests
import json
import re

# --- User input ---
keyword = input("Enter search keyword for Tiki: ").strip()
top_n = int(input("How many top products to fetch? (e.g. 3): ").strip() or "3")
reviews_per_product = int(input("How many reviews per product? (e.g. 10): ").strip() or "10")

# Use the same headers and cookies as in cell 1
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}
cookies = {
    'TIKI_GUEST_TOKEN': 'oC3QPaXrT4tZnw9jHfLiJhUvMmxSbEyN',
    'TOKENS': '{%22access_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1763654224277%2C%22guest_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj4bc.1k.11.2l',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj2q9.3.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605977607.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605977607.0',
    '_ga': 'GA1.2.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzkzizhzkzlznzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkzlzjzgzjzdzizizdzizlzjzmzqzkzlzjzgzjzdzizlzjzmzqzkzlzjzgzjzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605977598%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjAbsoluteSessionInProgress': '0',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605978058486',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    'cto_bundle': '7L6ha19NVXNkQmJ6aEVLcXNqbHdjcVZoQ0kzTUZwcEMyNCUyRm5nV3A2SThuOGxTRjI4Wlk1NU9xRnBEOG9tUjd2ekhyZEQxeE9qaVQ4MnFpbiUyRllGd2JiQUpTMW94MlNsTnYxd3dOYWtRcXhGdDNxSjdBVmNxU0FnUSUyQjlWYjhqTUtLdVl2cTBheWFvS0ZnY2pLdlpWRlEyUFF0Y1ElM0QlM0Q',
    'TIKI_RECENTLYVIEWED': '58259141',
    '_ants_event_his': '%7B%22action%22%3A%22view%22%2C%22time%22%3A1605977607258%7D',
    '_hjIncludedInPageviewSample': '1',
    '_hjIncludedInSessionSample': '1',
    '_gat': '1',
}

def get_tiki_reviews(keyword, top_n=5, reviews_per_product=40):
    # Step 1: Search for products
    search_url = 'https://tiki.vn/api/v2/products'
    search_params = {
        'q': keyword,
        'limit': top_n,
        'aggregations': '2'
    }
    try:
        search_response = requests.get(
            search_url,
            params=search_params,
            headers=headers,
            cookies=cookies
        )
        search_response.raise_for_status()
        products = search_response.json().get('data', [])
    except Exception as e:
        print(f"Search failed: {str(e)}")
        return {}

    # Step 2: Collect reviews for each product
    results = {}
    for product in products:
        product_id = product.get('id')
        product_name = product.get('name')
        if not product_id or not product_name:
            continue
        try:
            # Get reviews for this product
            reviews_url = 'https://tiki.vn/api/v2/reviews'
            reviews_params = {
                'product_id': product_id,
                'limit': reviews_per_product,
                'sort': 'score|desc'
            }
            reviews_response = requests.get(
                reviews_url,
                params=reviews_params,
                headers=headers,
                cookies=cookies
            )
            reviews_response.raise_for_status()
            review_data = reviews_response.json().get('data', [])
            # Only keep id and content for each review
            reviews_simple = [
                {'review_id': r.get('id', ''), 'content': r.get('content', '')}
                for r in review_data if r.get('content', '').strip()
            ][:reviews_per_product]
            results[product_name] = reviews_simple
            print(f"✅ Found {len(reviews_simple)} reviews for {product_name}")
        except Exception as e:
            print(f"⚠️ Failed to get reviews for product {product_id}: {str(e)}")
            results[product_name] = {'error': str(e)}
    return results

reviews_data = get_tiki_reviews(keyword, top_n=top_n, reviews_per_product=reviews_per_product)

# Save results to JSON file
with open(f'tiki_{keyword}_reviews.json', 'w', encoding='utf-8') as f:
    json.dump(reviews_data, f, ensure_ascii=False, indent=2)

print(f"\nResults saved to tiki_{keyword}_reviews.json")
print(f"Products found: {len(reviews_data)}")

# Debug: Show a sample of the collected reviews and file content
if reviews_data:
    sample_name = next(iter(reviews_data))
    print(f"\nSample product: {sample_name}")
    print("Sample reviews:", reviews_data[sample_name][:2])
else:
    print("No reviews collected.")

try:
    with open(f'tiki_{keyword}_reviews.json', encoding='utf-8') as f:
        print("\nFirst 500 chars of saved JSON file:")
        print(f.read(500))
except Exception as e:
    print(f"Could not read saved file: {e}")

✅ Found 20 reviews for Tai nghe Bluetooth Apple AirPods 4
✅ Found 20 reviews for Tai nghe Bluetooth Apple AirPods 3 cổng Lightning - MPNY3
✅ Found 17 reviews for Tai nghe Bluetooth Apple AirPods Pro 2 USB-C - MTJV3ZP/A
✅ Found 9 reviews for BAO CASE AIRPODS VỎ ỐP CHO TAI NGHE AIRPODS 1, AIRPODS 2, AIRPODS PRO
✅ Found 3 reviews for BAO CASE AIRPODS VỎ ỐP BẢO VỆ CHO TAI NGHE AIRPODS 1, AIRPODS 2, AIRPODS PRO

Results saved to tiki_airpod_reviews.json
Products found: 5

Sample product: Tai nghe Bluetooth Apple AirPods 4
Sample reviews: [{'review_id': 20136466, 'content': 'Đề nghị Tiki giải thích tại sao tai nghe Airpods mới nhận ngày 16/06/2025 và kích hoạt cùng ngày lại có bảo hành tới ngày 23/05/2026.'}, {'review_id': 20102243, 'content': 'Airpods mới dùng đã hư. Chính sách bảo hành lòng vòng. Cần nhân viên trả lời mà cứ đưa AI. Mất thời gian. Không uy tín.'}]

First 500 chars of saved JSON file:
{
  "Tai nghe Bluetooth Apple AirPods 4": [
    {
      "review_id": 20136466,
      "conte